In [38]:
!pip install openai-agents mcp uv kubectl_mcp_tool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [kubectl_mcp_tool][kubernetes]


In [1]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
#os.environ["OPENAI_API_KEY"] = ""

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"
#os.environ["OPENAI_MODEL"]="text-embedding-3-small"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

AZURE_OPENAI_API_KEY is ready


In [40]:
from agents import ModelSettings
from agents import OpenAIChatCompletionsModel
from dotenv import find_dotenv
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
from pydantic import BaseModel

# use azure openai api
openai_client = AsyncAzureOpenAI()

# configure the model
model = OpenAIChatCompletionsModel(
    os.getenv("OPENAI_MODEL", "gpt-4.1-mini"),
    openai_client=openai_client,
)

model_settings = ModelSettings(temperature=float(os.getenv("OPENAI_TEMPERATURE", 0.0)))

class Summary(BaseModel):
    reasoning: str
    summary: str
    insights: list[str]
    hashtags: list[str]

    def __str__(self) -> str:
        insights = "\n".join([f"  • {insight.strip()}" for insight in self.insights])
        hashtags = " ".join(self.hashtags)
        return "\n\n".join(
            [
                "📝 Summary",
                self.summary.strip(),
                "💡 Insights",
                insights,
                f"🏷️ Hashtags: {hashtags}",
            ]
        )

INSTRUCTIONS = """
    You are a helpful Slack bot assistant. When responding, you must
    strictly use Slack’s `mrkdwn` formatting syntax only. Do not generate
    headings (`#`), tables, or any other Markdown features not supported by
    Slack. Ensure that all output strictly complies with Slack’s `mrkdwn`
    specifications. Keep your responses readable and concise. Use English
    language for all responses. If you need to use Mandarin, please use
    Traditional Chinese (台灣繁體中文).
"""

from agents import Agent, Runner, trace
from agents.mcp import MCPServer, MCPServerStdio

async def run(mcp_server: MCPServer, input: str) -> None:
    result = await Runner.run(
        starting_agent=Agent(
            name="Assistant",
            instructions=INSTRUCTIONS.format(language="English", length=200),
            model=model,
            model_settings=model_settings,
            output_type=Summary,
            mcp_servers=[mcp_server]
        ),
        input=input,
    )
    print(result.final_output)

In [51]:

input = "Write a haiku about recursion in programming."

async def main():
    # Ask the user for the directory path

    async with MCPServerStdio(
        cache_tools_list=True,  # Cache the tools list, for demonstration
        client_session_timeout_seconds=10,
        # https://github.com/rohitg00/kubectl-mcp-server/blob/main/docs/QUICKSTART.md
        params={
            "command": "python3",
            "args": ["-m", "kubectl_mcp_tool.minimal_wrapper"],
            # https://github.com/rohitg00/kubectl-mcp-server/blob/main/README.md
            "env": {
                "PATH": "/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin",
                "MCP_TEST_MOCK_MODE": "1",
                #"MCP_LOG_FILE": "debug.log",
                "MCP_DEBUG": "1"
              }
        },
    ) as server:
        with trace(workflow_name="MCP Git Example"):
            await run(server, input)

await main()

Error initializing MCP server: Timed out while waiting for response to ClientRequest. Waited 10.0 seconds.
Error cleaning up server: unhandled errors in a TaskGroup (1 sub-exception)


McpError: Timed out while waiting for response to ClientRequest. Waited 10.0 seconds.